### New attempt
Old accuracy: 81,83026200325937

In [5]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import torch.optim as optim
from torch.nn import functional as F

import zipfile

# Path to the ZIP file
zip_file_path = 'train.csv.zip'

# Open the ZIP file and list its contents
with zipfile.ZipFile(zip_file_path, 'r') as z:
    # List all files in the ZIP archive
    file_list = z.namelist()
    print("Files in ZIP archive:", file_list)
    
    # Extract the specific CSV file
    csv_file_name = 'train.csv'
    with z.open(csv_file_name) as f:
        train_pd = pd.read_csv(f)

train_pd.head()

Files in ZIP archive: ['train.csv', '__MACOSX/._train.csv']


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47
0,9,0.253,0.303,0.353,0.37475,0.39650,0.41825,0.440,0.42425,0.40850,...,0.42400,0.41400,0.4040,0.394,0.420,0.391,0.3825,0.374,0.338,0.331
1,1,0.228,0.210,0.220,0.22167,0.22333,0.22500,0.275,0.24000,0.23233,...,0.53300,0.42000,0.3070,0.333,0.355,0.355,0.3520,0.349,0.370,0.391
2,9,0.281,0.244,0.278,0.33150,0.38500,0.35100,0.564,0.67100,0.67667,...,0.60033,0.57367,0.5470,0.479,0.505,0.475,0.4520,0.429,0.432,0.435
3,1,0.203,0.200,0.197,0.14400,0.18600,0.22800,0.233,0.20300,0.20267,...,0.53750,0.46000,0.3825,0.305,0.304,0.274,0.2595,0.245,0.197,0.149
4,14,0.289,0.369,0.449,0.43700,0.48600,0.53500,0.383,0.27000,0.25667,...,0.62900,0.64100,0.6530,0.610,0.635,0.511,0.4365,0.362,0.374,0.348


In [6]:
batch_size = 64
n_iters = 10000
num_epochs = 200
# Create CNN Model
n_timesteps = 46
n_outputs = 20

n = 199424 # number of samples less than 199424

y_np = train_pd.V1.values[:n]
x_np = train_pd.loc[:,train_pd.columns != "V1"].values[:n, :]
y_np = y_np - 1
# y goes from 0 to 19

x_train_np, x_test_np, y_train_np, y_test_np = train_test_split(x_np, y_np, test_size=0.2) 

x_train_tensor = torch.from_numpy(x_train_np).type(torch.FloatTensor)
y_train_tensor = torch.from_numpy(y_train_np).type(torch.LongTensor)

x_test_tensor = torch.from_numpy(x_test_np).type(torch.FloatTensor)
y_test_tensor = torch.from_numpy(y_test_np).type(torch.LongTensor)

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
test = torch.utils.data.TensorDataset(x_test_tensor, y_test_tensor)

train_loader = DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test, batch_size = batch_size, shuffle = False)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
class LSTMClassifier(nn.Module):
    """Very simple implementation of LSTM-based time-series classifier."""
    
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.batch_size = None
        self.hidden = None
    
    def forward(self, x):
        x = x.unsqueeze(1)
        h0, c0 = self.init_hidden(x)
        out, (hn, cn) = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return h0, c0

In [ ]:
input_dim = 46   
hidden_dim = 512
layer_dim = 3
output_dim = 20
seq_dim = 128

lr = 0.0005
n_epochs = 1000
iterations_per_epoch = 500
best_acc = 0
patience, trials = 100, 0

model = LSTMClassifier(input_dim, hidden_dim, layer_dim, output_dim)

criterion = nn.CrossEntropyLoss()
opt = torch.optim.RMSprop(model.parameters(), lr=lr)

print('Start model training')

for epoch in range(1, n_epochs + 1):
    
    for i, (x_batch, y_batch) in enumerate(train_loader):
        model.train()

        opt.zero_grad()
        out = model(x_batch)
        loss = criterion(out, y_batch)
        loss.backward()
        opt.step()
    
    model.eval()
    correct, total = 0, 0
    for x_val, y_val in test_loader:
        out = model(x_val)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_val.size(0)
        correct += (preds == y_val).sum().item()
    
    acc = correct / total

    if epoch % 500 == 0:
        print(f'Epoch: {epoch:3d}. Loss: {loss.item():.4f}. Acc.: {acc:2.2%}')

    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break

Start model training
Epoch 1 best model saved with accuracy: 68.61%
Epoch 2 best model saved with accuracy: 72.95%
Epoch 3 best model saved with accuracy: 74.49%
Epoch 4 best model saved with accuracy: 76.17%
Epoch 5 best model saved with accuracy: 77.58%
Epoch 6 best model saved with accuracy: 78.31%
Epoch 7 best model saved with accuracy: 78.75%
Epoch 8 best model saved with accuracy: 78.93%
Epoch 9 best model saved with accuracy: 79.30%
Epoch 10 best model saved with accuracy: 79.32%
Epoch 11 best model saved with accuracy: 79.57%
Epoch 12 best model saved with accuracy: 80.02%
Epoch 13 best model saved with accuracy: 80.24%
Epoch 14 best model saved with accuracy: 80.26%
Epoch 16 best model saved with accuracy: 80.60%
Epoch 17 best model saved with accuracy: 81.07%
Epoch 18 best model saved with accuracy: 81.11%
Epoch 19 best model saved with accuracy: 81.27%
